In [ ]:
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
%matplotlib qt5

In [ ]:
lon = np.linspace(-80, 80, 25)
lat = np.linspace(30, 70, 25)
lon2d, lat2d = np.meshgrid(lon, lat)

data = np.cos(np.deg2rad(lat2d) * 4) + np.sin(np.deg2rad(lon2d) * 4)

In [ ]:
# The projection keyword determines how the plot will look
plt.figure(figsize=(6, 3))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()

ax.contourf(lon, lat, data)  # didn't use transform, but looks ok...
plt.show()

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

In [ ]:
#ax = plt.axes(projection=ccrs.Mollweide())
ax.stock_img()

In [ ]:
# J1
#48°28'8.08"N
#1°29'26.72"W
J1 = [ 48 + 28 / 60 + 8.08 / 3600,
        -(1 + 29 / 60 + 26.72 / 3600) ]

# J2
#48°32'31.59"N
#1°35'15.44"W
J2 = [ 48 + 32 / 60 + 31.59 / 3600,
        -(1 + 35 / 60 + 15.44 / 3600) ]

# J11
#48°32'57.92"N
#1°34'22.25"W
J11 = [ 48 + 32 / 60 + 57.92 / 3600,
        -(1 + 34 / 60 + 22.25 / 3600) ]

# J12
#48°28'42.91"N
#1°28'33.18"W
J12 = [ 48 + 28 / 60 + 42.91 / 3600,
        -(1 + 28 / 60 + 33.18 / 3600) ]


In [ ]:
def main():
    # Create a Stamen Terrain instance.
    stamen_terrain = cimgt.StamenTerrain()

    fig = plt.figure()

    # Create a GeoAxes in the tile's projection.
    ax = fig.add_subplot(1, 1, 1, projection=stamen_terrain.crs)

    # Limit the extent of the map to a small longitude/latitude range.
    #ax.set_extent([-22, -15, 63, 65], crs=ccrs.Geodetic())
    ax.set_extent([-(1 + 35 / 60 + 15.44 / 3600), -(1 + 28 / 60 + 33.18 / 3600),
                   48 + 28 / 60 + 8.08 / 3600, 48 + 32 / 60 + 57.92 / 3600])
    ax.set_extent([-2, -1,
                   47, 49])

    # Add the Stamen data at zoom level 8.
    ax.add_image(stamen_terrain, 8)

    # Add a marker for the Eyjafjallajökull volcano.
    ax.plot(-19.613333, 63.62, marker='o', color='red', markersize=12,
            alpha=0.7, transform=ccrs.Geodetic())

    # Use the cartopy interface to create a matplotlib transform object
    # for the Geodetic coordinate system. We will use this along with
    # matplotlib's offset_copy function to define a coordinate system which
    # translates the text by 25 pixels to the left.
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
    text_transform = offset_copy(geodetic_transform, units='dots', x=-25)

    # Add text 25 pixels to the left of the volcano.
    ax.text(-19.613333, 63.62, u'Eyjafjallajökull',
            verticalalignment='center', horizontalalignment='right',
            transform=text_transform,
            bbox=dict(facecolor='sandybrown', alpha=0.5, boxstyle='round'))
    plt.show()


if __name__ == '__main__':
    main()

# GDAL Geographical Data Abstraction Layer

In [1]:
from osgeo import gdal,ogr
import numpy as np
import struct

In [3]:
root_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/"
src_filename = root_dir + "n48_w002_1arc_v3.dt2"

In [ ]:
def map2pixel(mx,my,gt):
    """
    Convert from map to pixel coordinates.
    Only works for geotransforms with no rotation.
    """

    px = int((mx - gt[0]) / gt[1]) #x pixel
    py = int((my - gt[3]) / gt[5]) #y pixel

    return px,py

In [ ]:
src_ds = gdal.Open(src_filename) 
gt = src_ds.GetGeoTransform()
rb = src_ds.GetRasterBand(1)

#For a single XY coordinate (must be same projection as the raster)
x = 123.456
y = 12.345
px , py = map2pixel(x,y,gt)
val = rb.ReadAsArray(px,py,1,1) #Read a 1x1 array from the raster at px py


## http://www.gdal.org/gdal_tutorial.html

In [4]:
dataset = gdal.Open(src_filename, gdal.GA_ReadOnly)

In [5]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))

# Fetch the coefficients for transforming between 
# 1) pixel/line (P,L) raster space, 
# 2) projection coordinates (Xp,Yp) space.
# Xgeo = GT(0) + Xpixel*GT(1) + Yline*GT(2)
# Ygeo = GT(3) + Xpixel*GT(4) + Yline*GT(5)
geotransform = dataset.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: DTED/DTED Elevation Raster
Size is 3601 x 3601 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
Origin = (-2.000138888888889, 49.000138888888884)
Pixel Size = (0.0002777777777777778, -0.0002777777777777778)


In [6]:
band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))
      
min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))
      
if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))
      
if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band Type=Int16
Min=-2.000, Max=357.000


In [7]:
band

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x7fa2987f22d0> >

In [13]:
scanline = band.ReadRaster(xoff = 0,
                           yoff = 0,
                           xsize = band.XSize, 
                           ysize = band.XSize,
                           buf_xsize = band.XSize, 
                           buf_ysize = band.XSize,
                           buf_type = gdal.GDT_Int32 ) # GDT_Int16 GDT_Float32

In [20]:
val = band.ReadAsArray(0, 0, band.XSize, band.XSize )

In [15]:
tuple_of_floats = struct.unpack('f' * band.XSize*band.YSize, scanline) # short h, float32 f

In [25]:
plt.figure()
plt.imshow(val, cmap="jet")
plt.grid()
plt.colorbar()

In [21]:
val.shape

(3601, 3601)

# Sentinel-1 tool box tests

In [1]:
import gdal

In [44]:
snap_dir = \
"/home/pleroy/DEV/SentinelsApplicationPlatform/RS2_OK76385_PK678063_DK606752_FQ2_20080415_143807_HH_VV_HV_VH_SLC/"
file = "imagery_VH.tif"

ds = gdal.Open(snap_dir + file)
band1 = ds.GetRasterBand(1)
band2 = ds.GetRasterBand(2)

In [46]:
print("Driver: {}/{}".format(ds.GetDriver().ShortName,
                             ds.GetDriver().LongName))
print("Size is {} x {} x {}".format(ds.RasterXSize,
                                    ds.RasterYSize,
                                    ds.RasterCount))
print("Projection is {}".format(ds.GetProjection()))

Driver: GTiff/GeoTIFF
Size is 2120 x 7863 x 2
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]


In [47]:
#<rasterAttributes>
    #<dataType>Complex</dataType>
    #<bitsPerSample dataStream="Real">16</bitsPerSample>
    #<bitsPerSample dataStream="Imaginary">16</bitsPerSample>
    #<numberOfSamplesPerLine>2120</numberOfSamplesPerLine>
    #<numberOfLines>7863</numberOfLines>
    #<sampledPixelSpacing units="m">4.73307896e+00</sampledPixelSpacing>
    #<sampledLineSpacing units="m">4.87164879e+00</sampledLineSpacing>
    #<lineTimeOrdering>Increasing</lineTimeOrdering>
    #<pixelTimeOrdering>Decreasing</pixelTimeOrdering>
#</rasterAttributes>

In [67]:
arr1 = band1.ReadAsArray().astype(float)
arr2 = band2.ReadAsArray().astype(float)

In [72]:
vmin = 0
vmax = 1000
plt.figure()
plt.imshow((arr1**2+arr2**2)**0.5, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.grid()

In [51]:
plt.figure()
plt.imshow(arr1, cmap="gray")

In [105]:
geoTransform = ds.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * ds.RasterXSize
miny = maxy + geoTransform[5] * ds.RasterYSize
print("minx={} miny={} maxx={} maxy={}".format(minx, miny, maxx, maxy))

minx=0.0 miny=7863.0 maxx=2120.0 maxy=0.0


# Try to create a GeoTIFF

## Get SRTM data

In [109]:
root_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/"
src_filename = root_dir + "n48_w002_1arc_v3.dt2"

dataset = gdal.Open(src_filename, gdal.GA_ReadOnly)

print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))

# Fetch the coefficients for transforming between 
# pixel/line (P,L) raster space => projection coordinates (Xp,Yp) space
# Xp = GT[0] + P*GT[1] + L*GT[2]
# Yp = GT[3] + P*GT[4] + L*GT[5]

GT = dataset.GetGeoTransform()
if GT:
    print("Origin = ({}, {})".format( GT[0], GT[3] ) )
    print("Pixel Size = ({}, {})".format( GT[1], GT[5] ) )

band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

XSize = band.XSize
YSize = band.YSize
dted_elevations = band.ReadAsArray(0, 0, XSize, YSize )

dted_long = GT[0] + np.arange(XSize) * GT[1]
dted_lat = GT[3] + np.arange(YSize) * GT[5]
meshgrid_long, meshgrid_lat = np.meshgrid(dted_long, dted_lat)

Driver: DTED/DTED Elevation Raster
Size is 3601 x 3601 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
Origin = (-2.000138888888889, 49.000138888888884)
Pixel Size = (0.0002777777777777778, -0.0002777777777777778)
Band Type=Int16


In [118]:
vmin = 0
vmax = 300
left = GT[0]
right = GT[0] + XSize * GT[1]
bottom = GT[3] + YSize * GT[5]
top = GT[3]

plt.figure()
plt.imshow(dted_elevations, extent=(left, right, bottom, top), vmin=vmin, vmax=vmax, cmap='terrain')
plt.colorbar()
plt.grid()
ax = plt.gca()

In [106]:
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

In [119]:
plt.figure()
plt.pcolormesh(dted_long, dted_lat, dted_elevations, vmin=vmin, vmax=vmax, cmap='terrain')

In [120]:
plt.savefig('ContourMapTight.png',dpi=900)

## Create GeoTIFF

In [123]:
from scipy import misc

In [125]:
img  = misc.imread('ContourMapTight.png')

In [127]:
plt.figure()
plt.imshow(img)